In [12]:
# Saving the Best Model...

#!pip install mlxtend
#!pip3 install numpy==1.24.0

In [26]:
# Backward Feature Selection method...


import pandas as pd
import numpy as np

#print(np.__version__)

import pickle
import matplotlib.pyplot as plt
import time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from mlxtend.feature_selection import SequentialFeatureSelector

#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import chi2
#from sklearn.feature_selection import RFE


1.24.0


In [14]:
def backward_selection(indep, dep,n):
    logistic_model=LogisticRegression(solver='saga', max_iter=10)
    
    bs = SequentialFeatureSelector(estimator=logistic_model, k_features=n, forward=False,floating=False,
                                    verbose=0,scoring=None,cv=5,n_jobs=1)
    
    fit1 = bs.fit(indep, dep)
    x_new= fit1.transform(indep)

    # get the selected feature names...
    selected_features=fit1.k_feature_names_
    
    return x_new, selected_features


def Split_Scaler(indep,dep):
    x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=0.30,random_state=0)
    scX=StandardScaler()
    x_train=scX.fit_transform(x_train)
    x_test=scX.transform(x_test)
    return x_train,x_test,y_train,y_test,scX


def cm_predition(classifier, x_test):
    y_pred=classifier.predict(x_test)

    #making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)

    # making Classification Report
    from sklearn.metrics import classification_report
    clf_report=classification_report(y_test,y_pred)

    # finding the accuracy score
    from sklearn.metrics import accuracy_score
    accuracy_score=accuracy_score(y_test,y_pred)
    
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred

def logistic(x_train,y_train,x_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state=0)
    classifier.fit(x_train,y_train)
    classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = cm_predition(classifier,x_test)
    return classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred 
    

# Displaying the values in the tabular frame

def BW_selection(accuracy_score):
    
    df=pd.DataFrame(index=["Logistic"],columns=["Logistic"])
    df["Logistic"]=accuracy_score
    
    return df




In [15]:
dataset=pd.read_csv("prep.csv",index_col=None)
df1=dataset  

#Convert categorical variable into dummy/indicator variables.
df1=pd.get_dummies(df1,drop_first=True,dtype=int)
df1

# separation of independent and dependent variables
indep=df1.drop("classification_yes",axis=1)
dep=df1["classification_yes"]
  
    
x_new, selected_features =backward_selection(indep, dep,5)

x_new

/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:73: FutureWarning: `fit_params` is deprecated and will be removed in version 1.6. Pass parameters via `params` instead.
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/venvaiml/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.

array([[  3.        , 148.11267606,  57.48210526,  12.51815562,
         38.86890244],
       [  2.        , 148.11267606,  22.        ,  10.7       ,
         34.        ],
       [  1.        ,  99.        ,  23.        ,  12.        ,
         34.        ],
       ...,
       [  3.        , 110.        , 115.        ,   9.1       ,
         26.        ],
       [  0.        , 207.        ,  80.        ,   8.5       ,
         38.86890244],
       [  0.        , 100.        ,  49.        ,  16.3       ,
         53.        ]])

In [16]:
x_train,x_test,y_train,y_test,scX = Split_Scaler(x_new, dep) 

classifier,cm,clf_report,accuracy_score,x_test,y_test,y_pred = logistic(x_train,y_train,x_test)

result= BW_selection(accuracy_score)

In [17]:
print("Accuracy: ",accuracy_score)

Accuracy:  0.9666666666666667


In [18]:
#result #5

In [25]:
# Save the Model
pickle.dump(classifier,open("finalized_logmodel_mlxtend",'wb'))

# save the Standard scalar object
pickle.dump(scX,open("scX_001.pkl",'wb'))

In [20]:
print(selected_features)


('al', 'bgr', 'bu', 'hrmo', 'pcv')


In [21]:
# mlxtend returned the result as Tuples. So, you can convert them to strings using the following approach:

selected_features = [''.join(feature)  for feature in selected_features]
print(selected_features)

['al', 'bgr', 'bu', 'hrmo', 'pcv']


In [22]:
df1[selected_features]

,al,bgr,bu,hrmo,pcv
0,3.0,148.112676,57.482105,12.518156,38.868902
1,2.0,148.112676,22.000000,10.700000,34.000000
2,1.0,99.000000,23.000000,12.000000,34.000000
3,1.0,148.112676,16.000000,8.100000,38.868902
4,0.0,148.112676,25.000000,11.800000,36.000000
...,...,...,...,...,...
394,0.0,219.000000,36.000000,12.500000,37.000000
395,0.0,220.000000,68.000000,8.700000,27.000000
396,3.0,110.000000,115.000000,9.100000,26.000000
397,0.0,207.000000,80.000000,8.500000,38.868902
